# Part 4: Quantization

In [5]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf

tf.random.set_seed(seed)
import os

os.environ['PATH'] = '/tools/Xilinx/Vitis/2022.1/bin:' + os.environ['PATH']
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import tensorflow as tf

In [7]:
df_train = pd.read_csv('./input/X_train.txt', delimiter =r'\s+(?=-)|\s+', header=None)
df_train.head()
X_train = df_train.values
print(X_train.shape)

/tmp/ipykernel_82300/1605945988.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv('./input/X_train.txt', delimiter =r'\s+(?=-)|\s+', header=None)


(7352, 561)


In [8]:
X_train = df_train.values
print(X_train.shape)

(7352, 561)


In [9]:
dfyt = pd.read_csv('./input/y_train.txt', delimiter=' ', header=None)
dfyt = dfyt.rename(index = int,columns = {0:561})
Y_train = dfyt.values
Y_train -= 1
print(Y_train.shape)

(7352, 1)


In [10]:
le = LabelEncoder()
print(Y_train)
Y_train = le.fit_transform(Y_train)
print(le.classes_)
Y_train = to_categorical(Y_train, 6)
print(Y_train)
print(Y_train.shape)

[[4]
 [4]
 [4]
 ...
 [1]
 [1]
 [1]]
[0 1 2 3 4 5]
[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]
(7352, 6)


/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
df_test = pd.read_csv('./input/X_test.txt', delimiter =r'\s+(?=-)|\s+', header=None)
df_test.head()
X_test = df_test.values
print(X_test.shape)

/tmp/ipykernel_82300/2884645286.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test = pd.read_csv('./input/X_test.txt', delimiter =r'\s+(?=-)|\s+', header=None)


(2947, 561)


In [12]:
X_test = df_test.values
print(X_test.shape)

(2947, 561)


In [13]:
y_test = pd.read_csv('./input/y_test.txt', delimiter=' ', header=None)
dfytest = y_test.rename(index = int,columns = {0:561})
Y_test = dfytest.values
Y_test -= 1
print(Y_test.shape)

(2947, 1)


In [14]:
le = LabelEncoder()
print(Y_test)
Y_test = le.fit_transform(Y_test)
print(le.classes_)
Y_test = to_categorical(Y_test, 6)
print(Y_test)

[[4]
 [4]
 [4]
 ...
 [1]
 [1]
 [1]]
[0 1 2 3 4 5]
[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",Y_train.shape)
print("Shape of y_test",Y_test.shape)

Shape of X_train:  (7352, 561)
Shape of X_test:  (2947, 561)
Shape of y_train:  (7352, 6)
Shape of y_test (2947, 6)


## Construct a model
This time we're going to use QKeras layers.
QKeras is "Quantized Keras" for deep heterogeneous quantization of ML models.

https://github.com/google/qkeras

It is maintained by Google and we recently added support for QKeras model to hls4ml.

In [16]:
# from sklearn.neural_network import MLPClassifier
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# clf = MLPClassifier(beta_1=0.5553817511197982, beta_2=0.7408615040451714,
#               epsilon=0.769773517554749, hidden_layer_sizes=(64,128,64),
#               learning_rate='adaptive', max_iter=150,
#               momentum=0.9205475833293032, solver='lbfgs',
#               validation_fraction=0.7720568212000914).fit(X_train, Y_train)
# clf.score(X_test, Y_test)

In [2]:
import tensorflow as tf
import os
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# int_bits=1
# model = tf.keras.Sequential()
# model.add(QDense(20, input_shape=(561,), name='fc1', kernel_quantizer=quantized_bits(8,int_bits,alpha=1,use_stochastic_rounding=True),bias_quantizer=quantized_bits(8,int_bits,alpha=1),
#                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)   ) )
# model.add(QActivation(activation=quantized_relu(8,int_bits,use_stochastic_rounding=False), name='relu1'))
# model.add(QDense( 64, name='fc2',
#                 kernel_quantizer=quantized_bits(8,int_bits,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(8,int_bits,alpha=1),
#                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
# model.add(QActivation(activation=quantized_relu(8,int_bits,use_stochastic_rounding=False), name='relu2'))
# model.add(QDense(64 ,name='fc3',
#                 kernel_quantizer=quantized_bits(8,int_bits,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(8,int_bits,alpha=1),
#                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
# model.add(QActivation(activation=quantized_relu(8,int_bits,use_stochastic_rounding=False), name='relu3'))
# model.add(QDense( 6, name='output',
#                 kernel_quantizer=quantized_bits(8,int_bits,alpha=1,use_stochastic_rounding=True), bias_quantizer=quantized_bits(8,int_bits,alpha=1),
#                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001 ) ))
# model.add(Activation(activation='softmax', name='softmax'))

In [ ]:
# from tensorflow.keras.optimizers import Adam
# from callbacks import all_callbacks
# adam = Adam(lr=0.01)
# model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
# callbacks= all_callbacks( outputDir = 'HAR_test')
# #callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
# model.fit(X_train, Y_train, batch_size=128,
#               epochs=100,validation_split=0.2, verbose=1, shuffle=True,
#               callbacks = callbacks.callbacks);
# model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
# model_save_quantized_weights(model, "test_weights")

# model.evaluate(X_test,Y_test)
# model.save("HAR_model")

In [3]:
model = tf.keras.models.load_model("HAR_model")

2023-07-19 00:04:08.247848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
accuracy=model.evaluate(X_test,Y_test)
print(" ACCURACY IS "+str(accuracy[1]) )

93/93 [==============================] - 1s 4ms/step - loss: 0.2689 - accuracy: 0.9433
 ACCURACY IS 0.9433321952819824


In [18]:
layer_1=561
layer_2=20
layer_3=64
layer_4=64
layer_5=6
int_bits=1
sign_bit=1
bits=8

In [31]:
we=model.get_weights()[0]

In [32]:
we

array([[-0.078125, -0.03125 ,  0.09375 , ..., -0.015625,  0.03125 ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [ 0.015625, -0.09375 , -0.15625 , ...,  0.03125 ,  0.      ,
         0.      ],
       [-0.03125 ,  0.234375,  0.015625, ..., -0.046875,  0.015625,
         0.      ],
       [ 0.      ,  0.09375 ,  0.046875, ...,  0.      ,  0.046875,
         0.      ]], dtype=float32)

In [38]:
num_inp=np.sum(we==0,axis=1)
total_zero_weights = np.sum(num_inp)
num_inputs_with_zero_weights=np.where(num_inp == we.shape[1])[0]
num_inputs = len(num_inputs_with_zero_weights)
print("Num of inputs with only zeros= ",num_inputs)

Num of inputs with only zeros=  0


In [19]:
import estimate as es

In [29]:
from importlib import reload
reload(es)

<module 'estimate' from '/home/edge/Desktop/argykokk/hls4ml-tutorial/networks/HAR/estimate.py'>

In [25]:
param=2
reuse=100

In [30]:
%%time
#input_num, neurons_num, layer_id, model
luts, ffs = es.estimate(561,20,0,model,reuse,param)
luts, ffs = es.estimate(20,64,2,model,reuse,param)
luts, ffs = es.estimate(64,64,4,model,reuse,param)
luts, ffs = es.estimate(64,6,6,model,reuse,param)

zero weight are:  4381
non-zero weight are:  6839
Mul ins = 33 and Max muls = 69 and Saved muls = 0 Reuse factor = 100
Muxes LUTS: 0
DSPs: 33
LUT cost2= 33170 bias acc= 280 mult acc95466
LUTs prediction: 128916
Initial muls: 11220 Real muls: 6839 Initial neurons: 20 Tuned neurons: 12
FFs prediction: 131000
zero weight are:  472
non-zero weight are:  808
Mul ins = 156 and Max muls = 9 and Saved muls = 147 Reuse factor = 100
Muxes LUTS: 1911
DSPs: 9
LUT cost2= 4712 bias acc= 896 mult acc10416
LUTs prediction: 16024
Initial muls: 1280 Real muls: 808 Initial neurons: 64 Tuned neurons: 40
FFs prediction: 15336
zero weight are:  2130
non-zero weight are:  1966
Mul ins = 129 and Max muls = 20 and Saved muls = 109 Reuse factor = 100
Muxes LUTS: 1417
DSPs: 20
LUT cost2= 10930 bias acc= 896 mult acc26628
LUTs prediction: 38454
Initial muls: 4096 Real muls: 1966 Initial neurons: 64 Tuned neurons: 30
FFs prediction: 36379
zero weight are:  86
non-zero weight are:  298
Mul ins = 89 and Max muls = 3

/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/edge/miniconda3/envs/hls4ml-tutorial/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.1.2 when 

In [ ]:
import hls4ml
import plotting

config = hls4ml.utils.config_from_keras_model(model, granularity='name')
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'
print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='HAR/hls4ml_prj', part='xc7z007s-clg225-2'
)
hls_model.compile()

y_qkeras = model.predict(np.ascontiguousarray(X_test))
y_hls = hls_model.predict(np.ascontiguousarray(X_test))